In [3]:
import requests 
import gzip, shutil
import os 
import data_utils as du 
import tensorflow as tf 
from tensorflow import keras 
import numpy as np 
from six.moves import cPickle as pickle
import numpy as np
import os
from matplotlib.pyplot import imread
import matplotlib.pyplot as plt 
import platform
%load_ext autoreload
%autoreload 2
%matplotlib inline

plt.rcParams['figure.figsize'] = (15.0, 15.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

Data_folder = 'CIFAR10_Data'
Data_fn = 'cifar-10-python'
cifar_url = 'https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz'
#Chang to true if need download data 
Down_load_data = False 
if Down_load_data:
    r = requests.get(cifar_url,allow_redirects=True)
    open(os.path.join(Data_folder,Data_fn+'.tar.gz'),'wb').write(r.content)
    #Unzip the data

In [4]:
import tensorflow as tf 
from tensorflow import keras 
import numpy as np 
from six.moves import cPickle as pickle
import numpy as np
import os
from matplotlib.pyplot import imread
import platform

def load_pickle(f):
  version = platform.python_version_tuple()
  if version[0] == '2':
      return  pickle.load(f)
  elif version[0] == '3':
      return  pickle.load(f, encoding='latin1')
  raise ValueError("invalid python version: {}".format(version))

def load_CIFAR_batch(fn):
    with open(fn, 'rb') as f:
        datadict = load_pickle(f)
        X = datadict['data']
        Y = datadict['labels']
        X = X.reshape(10000, 3, 32, 32).transpose(0,2,3,1).astype("float")
        Y = np.array(Y)
        return X, Y

def load_CIFAR10(ROOT):
    """ load all of cifar """
    xs = []
    ys = []
    for b in range(1,6):
        f = os.path.join(ROOT, 'data_batch_%d' % (b, ))
        X, Y = load_CIFAR_batch(f)
        xs.append(X)
        ys.append(Y)
    Xtr = np.concatenate(xs)
    Ytr = np.concatenate(ys)
    del X, Y
    Xte, Yte = load_CIFAR_batch(os.path.join(ROOT, 'test_batch'))
    return Xtr, Ytr, Xte, Yte

def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=10000,subtract_mean=True):
    """
    Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
    it for training  
    """
    # Load the raw CIFAR-10 data
    cifar10_dir = 'CIFAR10_Data/cifar-10-batches-py'
    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]
    
    ## Data post_process 
   
    if subtract_mean:
        mean_image = np.mean(X_train, axis=0)
        X_train -= mean_image
        X_val -= mean_image
        X_test -= mean_image
    return X_train, y_train, X_val, y_val, X_test, y_test

In [5]:
X_train, y_train, X_val, y_val, X_test, y_test = get_CIFAR10_data(subtract_mean =True)
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Train data shape:  (49000, 32, 32, 3)
Train labels shape:  (49000,) int64
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


In [6]:
from solver import Solver
from cnn_model import *
import resent2 
import resnet
#model = ThreeLayerConvNet(12, 8, 10)
#model = Lennet5(in_channel=3, out_channel=10)
#
model = resent2.cifar_resnet20('preactivated', shortcut_type='B')
#model = resnet.ResNet20(3,10)
data_set = {
    'train_data': X_train,
    'train_label': y_train,
    'val_data':X_val,
    'val_label':y_val,
}
train_options = {
    'optimizer': 'adam',
    'lr': 0.001,
    'epoch_num': 20,
    'save_dir': 'complier_tmp',
    'from_logits': True
}

#model.build((1,32,32,3))
#print(model.summary())
solver= Solver(model,data_set,options=train_options)
#solver.train()





Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 16)   432         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 16)   64          conv2d[0][0]                     
__________________________________________________________________________________________________
re_lu (ReLU)                    (None, 32, 32, 16)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

## load model 

In [7]:
import tensorflow_hub as hub
from solver import Solver
import quantize_util as qu 
model_path = 'complier_tmp/trained_15'
model = hub.KerasLayer(model_path,trainable=True)
solver= Solver(model,data_set,options=train_options)
test_loss, test_acc = solver.test_model(X_test,y_test)
model_q = resent2.cifar_resnet20('preactivated', shortcut_type='B')
qu.copy_weight(model_q,model)
solver= Solver(model_q,data_set,options=train_options)
test_loss, test_acc = solver.test_model(X_test,y_test)


Loss: 1.0970335006713867,Test Accuracy: 65.97000122070312
Loss: 1.0970335006713867,Test Accuracy: 65.97000122070312


In [8]:
for var in model_q.trainable_variables:
    print(var.name)


conv2d_42/kernel:0
batch_normalization_39/gamma:0
batch_normalization_39/beta:0
conv2d_43/kernel:0
batch_normalization_40/gamma:0
batch_normalization_40/beta:0
conv2d_44/kernel:0
batch_normalization_41/gamma:0
batch_normalization_41/beta:0
conv2d_45/kernel:0
batch_normalization_42/gamma:0
batch_normalization_42/beta:0
conv2d_46/kernel:0
batch_normalization_43/gamma:0
batch_normalization_43/beta:0
conv2d_47/kernel:0
batch_normalization_44/gamma:0
batch_normalization_44/beta:0
conv2d_48/kernel:0
batch_normalization_45/gamma:0
batch_normalization_45/beta:0
conv2d_49/kernel:0
batch_normalization_46/gamma:0
batch_normalization_46/beta:0
conv2d_51/kernel:0
conv2d_50/kernel:0
batch_normalization_47/gamma:0
batch_normalization_47/beta:0
conv2d_52/kernel:0
batch_normalization_48/gamma:0
batch_normalization_48/beta:0
conv2d_53/kernel:0
batch_normalization_49/gamma:0
batch_normalization_49/beta:0
conv2d_54/kernel:0
batch_normalization_50/gamma:0
batch_normalization_50/beta:0
conv2d_55/kernel:0
ba

In [9]:
import quantize_util as qu 
val = qu.to_fixpoint(4.2,wl=8,fl=-3)
print(val)

tf.Tensor(8.0, shape=(), dtype=float32)


In [19]:
import quantize_complier as qc 
fl_agent = qc.FlSerachAgent(model,solver,[X_test,y_test])
test_weight =  model_q.trainable_variables[1].numpy().reshape(-1)
min = -32
fls = [*range(min,32)]
for i in range(1,32):
    errors = fl_agent.quantize_abs_mean_errors(test_weight,i,fls)
    #print('min error at: {}'.format(np.argmin(errors)+min))
    #print(np.min(errors))






In [20]:
for i in range(1,32):
    best_fl = fl_agent.search_fl(test_weight,i)
    print(best_fl)



0
0
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
26
27
27
28


In [21]:
solver = Solver(model,)
def tunner():
    
    print("Tune the model")
fl_agent = qc.FlSerachAgent(model,tunner,1)
fl_agent.tunne_model()


Tune the model


## Test for FL_agent function

In [60]:
tune_size = 40000
X_tune = X_train[0:tune_size]
y_tune = y_train[0:tune_size]
tune_dataset = {
    'batch_size': 32,
    'train_data': X_tune,
    'train_label': y_tune,
    'val_data':X_val,
    'val_label':y_val,
}
tune_options = {
    'optimizer': 'adam',
    'lr': 0.001,
    'epoch_num': 1,
    'from_logits': True
}

tune_solver = Solver(model,tune_dataset,save_model=False,                               options=tune_options,plot_graph=False)
def tune_fn():
    tune_solver.train()
def acc_fn():
    loss,acc = tune_solver.test_model(X_test,y_test)
    return acc.numpy() 

fl_agent = qc.FlSerachAgent(model,tune_fn,acc_fn)
fl_agent.tunne_model()
fl_agent.model_accuracy()

Epoch: 0/1
 1250/1250 |=========================|  - train_acc: 0.7416 - train_loss: 0.7569 - val_acc: 0.6960 - val_loss: 0.9562
Loss: 1.0654171705245972,Test Accuracy: 65.56999969482422


0.6557